In [ ]:
import pandas as pd

In [ ]:
data_ari=pd.read_csv("/content/100_clickbait.csv")
data_kaus=pd.read_csv("/content/Youtube_ARi_100.csv")

data_mer=data_ari+data_kaus
data=pd.DataFrame(data_mer)

data=data.drop(['Unnamed: 0'],axis=1)
data.columns

Index(['title', 'isClickbait', 'channelTitle', 'view_count', 'likes',
       'dislikes', 'comment_count', 'description'],
      dtype='object')

In [ ]:
class_list = [0, 1]
data = data[['description','isClickbait']]

#to change to 1 where isClickbait is 2
data.loc[data["isClickbait"] == 2, "isClickbait"] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
data

,description,isClickbait
0,Watch tomorrow’s Premiere Episodes of ZEE TV s...,0
1,RG Music Factory Presents Brand New Song KANGA...,0
2,Mr Detective 2 - Android - https://bit.ly/MrDe...,0
3,"During Mann Ki Baat, Prime Minister Narendra M...",0
4,"In this video, we test that how much weight ca...",0
...,...,...
94,ഈ കാന്താരിപ്പെണ്ണിന്റെ മനസ്സ് കീഴടക്കി ശിവൻ.. ...,1
95,Subscribe My Main Channel 🙏🙏🤗🤗 Watch All My Vi...,1
96,Dhani app use karo aur Dhani bano!Click on the...,0
97,follow us on Instagram - https://www.instagram...,1


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size=0.10)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

# train shape:  (6309, 2)
# test shape:  (702, 2)

train shape:  (89, 2)
test shape:  (10, 2)


In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "albert", "ai4bharat/indic-bert",
    num_labels=2,
    args=train_args,
    use_cuda=False
)

# Tell pytorch to run this model on the GPU.
#desc = model.cuda()

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['sop_classifier.classifier.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

In [ ]:
model.train_model(train_df)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

# {'acc': 0.6894586894586895,
# 'eval_loss': 0.8673831869594075,
# 'f1': 0.6894586894586895,
# 'mcc': 0.25262380289641617}

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/10 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
result

{'acc': 0.5,
 'auprc': 0.4746031746031746,
 'auroc': 0.32000000000000006,
 'eval_loss': 0.7066702842712402,
 'f1': 0.5,
 'fn': 0,
 'fp': 5,
 'mcc': 0.0,
 'tn': 0,
 'tp': 5}

In [ ]:
import os
import tarfile

def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

# run the function
pack_model('./','model')

In [ ]:
test_title = " 1 min. Easy Beautiful Hair Bun Hairstyle"
 
predictions, raw_outputs = model.predict([test_title])

print(test_title)
print(raw_outputs)
 
print(predictions[0])
#print(class_list[predictions[0]])
# OTHER


test_title2 = "लाल साड़ी में भाभी ने किया ऐसा जोरदार डांस, वीडियो से नहीं हटेंगी आपकी नज़र"
 
predictions, raw_outputs = model.predict([test_title2])
print(test_title2)
print(raw_outputs)
print(predictions[0])


test_title3 = "கோக-கோலா நிறுவனத்தை வாங்குகிறாரா எலான் மஸ்க்?"
 
predictions, raw_outputs = model.predict([test_title3])
print(test_title3)
print(raw_outputs)
print(predictions[0])


test_title4 = "বিজয় বাবুর বিরুদ্ধে ধর্ষণের অভিযোগ, সোশ্যাল মিডিয়ায় নির্যাতিতার পরিচয় ফাঁস, বিপাকে মালায়ালি অভিনেতা"
 
predictions, raw_outputs = model.predict([test_title4])
print(test_title4)
print(raw_outputs)
print(predictions[0])



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

 1 min. Easy Beautiful Hair Bun Hairstyle
[[-0.04991629  0.01121246]]
1


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

கோக-கோலா நிறுவனத்தை வாங்குகிறாரா எலான் மஸ்க்?
[[-0.05604164  0.01538202]]
1


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

বিজয় বাবুর বিরুদ্ধে ধর্ষণের অভিযোগ, সোশ্যাল মিডিয়ায় নির্যাতিতার পরিচয় ফাঁস, বিপাকে মালায়ালি অভিনেতা
[[-0.05488615  0.02001617]]
1
